# Script d'insertion des données dans la table "games"

In [ ]:
import pandas as pd
import mysql.connector

In [ ]:
from config import config

In [ ]:
descriptions_path = "../data/games_description.csv"

df_descriptions = pd.read_csv(descriptions_path)

In [ ]:
df_descriptions

### Préparation du dataframe pour insertion dans la table

In [ ]:
# On construit un nouveau dataframe à partir des attributs qui nous intéressent 

df_games = df_descriptions[['name', 'short_description', 'long_description', 'release_date', 'overall_player_rating', 'number_of_reviews_from_purchased_people', 'number_of_english_reviews', 'link']]

In [ ]:
# On renomme les colonnes pour correspondre à celles dans la table MySQL

df_games = df_games.rename(columns={'short_description': 'short_desc', 'long_description': 'long_desc'})

In [ ]:
# On crée la colonne "price", valeurs nulles pour l'instant

df_games['price'] = None

In [ ]:
# On fait correspondre l'ordre des colonnes du dataframe avec celles de la table

df_games = df_games[['name', 'release_date', 'link', 'short_desc', 'long_desc', 'price', 'number_of_english_reviews', 'number_of_reviews_from_purchased_people', 'overall_player_rating']]

### Transformation et normalisation des données

In [ ]:
# Interprétation et conversion des dates en format date (chaînes de caractères dans le .csv)

df_games['release_date'] = pd.to_datetime(df_games['release_date'], format='mixed', errors='coerce')
df_games['release_date'] = df_games['release_date'].dt.date

In [ ]:
# Conversion des nombres (chaînes de caractères ici encore) en entiers, en supprimant les virgules et espaces

df_games['number_of_english_reviews'] = (
    df_games['number_of_english_reviews']
    .astype(str)
    .str.replace(',', '')
    .replace('', None)
    .astype(float)
    .astype('Int64')
)

In [ ]:
# Conversion des valeurs Nan, NaT en None (préférable en base de données)

df_games = df_games.where(pd.notnull(df_games), None)

### Insertion des données

In [ ]:
# Requête SQL

insert_query = """
    INSERT INTO games
    (name, release_date, link, short_desc, long_desc, price, number_of_english_reviews, number_of_reviews_from_purchased_people, overall_player_rating)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

In [ ]:
# Connexion à la base et exécution de la requête

cnx = mysql.connector.connect(**config)
cursor = cnx.cursor()

for _, row in df_games.iterrows():
    cursor.execute(insert_query, tuple(row))

cnx.commit()
cursor.close()
cnx.close()